In [ ]:
# Outputwidget zum Debuggen von Callbacks
# SPlite, behandle auch canvas Events
# copy ev. allg text zu widgets

### Das Output-Widget
Siehe auch [Doku zum Output-Widget](https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html).  

Das Output-Widget ist ein Fenster, in das sich Output, meist Text, umleiten l&auml;sst.
```python
# Widget erstellen
out = Output(layout={'border': '1px solid black'})

# aller vom Codeblock ... produzierter Output,
# inkl. Fehlermeldungen nach out umleiten
with out:
    ...

# aller von f(...) produzierter Output nach out umgeleiten
@out.capture()
def f(...):
    ...

# vor jedem Aufruf von f(...) wird der Inhalt von out geloescht
@out.capture(clear_output=True)
def f(...):
    ...
```

In [16]:
from ipywidgets import Output

In [ ]:
# ohne den schwarzen Rand ist ein leeres Output-Widget unsichtbar
layout = {'border': '1px solid black'}
out = Output(layout=layout)
out

In [ ]:
# Output in out loeschen
out.clear_output()

In [14]:
def new_out(**kwargs):
    layout = {'border': '1px solid black'}
    for k, v in kwargs.items():
        layout[k] = v
    out = Output(layout=layout)
    return out

In [ ]:
with out:
    print('Hello!')

In [ ]:
with out:
    # Fehlermeldung wird nach out umgeleitet
    print(foo)

In [ ]:
# aller Output von f wird nach out umgeleitet
@out.capture()
def f():
    print('Hello from f')


# aller Output von f wird nach out umgeleitet, vorher wird out geleert
@out.capture(clear_output=True)
def g():
    print('Hello from g')

In [ ]:
f()

In [ ]:
g()

### Fehlermeldungen von Callbacks in ein Output-Widget umleiten

Ruft ein Jupyterlab-Widget eine Callback-Funktion auf,
so wird deren Output (inkl.Fehlermeldugen) nicht direkt ins 
Notebook geschreiben, sondern (wenn man Gl&uuml;ck hat) in die Log-Console. Ruft z.B. ein Button-Widget beim 
Draufklicken ein fehlerhaftes Callback auf, passiert oft einfach gar nichts, was das auffinden des Fehlers sehr schwierig macht.

Aus diesem Grund empfiehlt es sich,
ein Output-Widget f&uuml;r die Ausgabe der Fehler zu erstellen und allen Output der Callbacks dorthin umzuleiten.

```python
err_out = new_out()

@err_out.capture()
def some_callback():
    ...
```

In [ ]:
err_out = Output(layout=layout)

In [ ]:
@err_out.capture()
def on_key_down(key, *flags):
    print(key, flag)


@err_out.capture()
def on_mouse_down(x, y):
    print('on_mouse_down', x, y)


@err_out.capture()
def on_mouse_up(x, y):
    print('on_mouse_up', x, y)

In [ ]:
from ipycanvas import Canvas


canvas_config = {
    'width': 300,
    'height': 200,
    'layout': layout,
}

canvas = Canvas(**canvas_config)

canvas.on_key_down(on_key_down)
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_up(on_mouse_up)

display(canvas, err_out)

In [ ]:
err_out.clear_output()

In [9]:
# NIM Game

heaps = [9, 10, 11]


def print_heaps(heaps):
    for i, n in enumerate(heaps):
        s = '***** ' * (n // 5) + '*' * (n % 5)
        print('{}) {}'.format(i, s))
    print()


def update(heaps, idx, n):
    heaps[idx] = max(0, heaps[idx]-n)


def best_nim_move(heaps):
    tot = sum(heaps)
    for i, heap in enumerate(heaps):
        if heap:
            nonempty = i
        n = heap ^ tot
        if n < heap:
            move = (i, heap - n)
            break
    else:
        move = (nonempty, 1)
    return move


def move(idx, n):
    update(heaps, idx, n)
    print(f'You removed {n} from heap {idx}')
    print_heaps(heaps)
    if sum(heaps) == 0:
        print('You won!')

    idx, n = best_nim_move(heaps)
    print(f'Compi removed {n} from heap {idx}')
    update(heaps, idx, n)
    print_heaps(heaps)
    if sum(heaps) == 0:
        print('You lost!')

In [10]:
print_heaps([12, 3, 5])

0) ***** ***** **
1) ***
2) ***** 



In [11]:
move(0, 10)

You removed 10 from heap 0
0) 
1) ***** ***** 
2) ***** ***** *

Compi removed 1 from heap 2
0) 
1) ***** ***** 
2) ***** ***** 



In [12]:
from ipywidgets import Text


# Deprecation Warnings unterdruecken
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [17]:
out_main = new_out(height='200px')
out_err = new_out()

heaps = [9, 10, 11]
prompt = 'Haufen, wieviele: '


@out_err.capture()
def on_press_enter(text):
    idx, n = [int(s) for s in text.value.removeprefix(prompt).split(',')]
    out_main.clear_output()
    with out_main:
        move(idx, n)

    text.value = prompt


text = Text(value=prompt)
text.on_submit(on_press_enter)

In [18]:
with out_main:
    print_heaps(heaps)

display(text, out_main, out_err)
text.focus()

Text(value='Haufen, wieviele: ')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [90]:
out_main = new_out(height='200px')
out_err = new_out()

heaps = [9, 10, 11]
prompt = 'Haufen, wieviele: '
out_main = new_out(height='200px')


@out_err
def on_press_enter(text):
    idx, n = [int(s) for s in text.value.removeprefix(prompt).split(',')]

    update(heaps, idx, n)
    print(f'You removed {idx} from heap {n}')
    print_heaps(heaps)

    print('Compi')
    idx, n = ask_compi(heaps)
    update(heaps, idx, n)
    print_heaps(heaps)

    text.value = prompt


text = Text(value=prompt)
text.on_submit(on_press_enter)

with out_main:
    print_heaps(heaps)

display(text, out_main)
text.focus()

Text(value='Haufen, wieviele: ')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…